# 抽象文档模式

## 目标

使用Java这种静态语言获得动态语言的灵活性

## 应用场景

- 需要动态添加新属性
- 希望更灵活的组织树状的结构的数据
- 让系统更加松耦合

![ORM](https://github.com/iluwatar/java-design-patterns/raw/master/abstract-document/etc/abstract-document.png)

## 来源

[abstract-document](https://github.com/iluwatar/java-design-patterns/tree/master/abstract-document)

In [1]:
/**
 * 定义Document接口
 */
import java.util.Map;
import java.util.function.Function;
import java.util.stream.Stream;

public interface Document {
    void put(String key, Object val); //设置属性
    Object get(String key); //获取属性
    //获得子节点文档的Stream
    <T> Stream<T> children(String key, Function<Map<String, Object>, T> constructor);
}

In [2]:
/**
 * 定义AbstractDocument抽象类
 */
import java.util.List;
import java.util.Map;
import java.util.Objects;
import java.util.Optional;
import java.util.function.Function;
import java.util.stream.Stream;

public abstract class AbstractDocument implements Document {
    private final Map<String, Object> properties;
    protected AbstractDocument(Map<String, Object> properties) {
        Objects.requireNonNull(properties, "properties can't be null");
        this.properties = properties;
    }
    
    @Override
    public void put(String key, Object val) {
        properties.put(key, val);
    }
    
    @Override
    public Object get(String key) {
        return properties.get(key);
    }
    
    @Override
    public <T> Stream<T> children(String key, Function<Map<String, Object>, T> constructor) {
        Optional<List<Map<String, Object>>> any = Stream.of(get(key)).filter(e -> e != null).
            map(e -> (List<Map<String, Object>>) e).findAny();
        return any.isPresent() ? any.get().stream().map(constructor) : Stream.empty();
    }
    
    @Override
    public String toString() {
        StringBuilder sb = new StringBuilder();
        sb.append(getClass().getName()).append("[");
        properties.entrySet().forEach( e -> sb.append("[").append(e.getKey()).
            append(" : ").append(e.getValue()).append("]"));
        sb.append("]");
        return sb.toString();
    }
}

In [3]:
/**
 * 定义一些属性接口
 */

import java.util.Optional;

public interface HasModel extends Document {
    String PROPERTY = "model";
    
    default Optional<String> getModel() {
        return Optional.ofNullable((String) get(PROPERTY));
    }
}

public interface HasType extends Document {
    String PROPERTY = "type";
    
    default Optional<String> getType() {
        return Optional.ofNullable((String) get(PROPERTY));
    }    
}

public interface HasPrice extends Document {
    String PROPERTY = "price";
    
    default Optional<Number> getPrice() {
        return Optional.ofNullable((Number) get(PROPERTY));
    }
}

In [4]:
/**
 * 定义部件类
 */

import java.util.Map;

public class Part extends AbstractDocument implements HasType, HasModel, HasPrice {
    public Part(Map<String, Object> properties) {
        super(properties);
    }
}

In [5]:
/**
 * 定义部件属性接口
 */

import java.util.Optional;
import java.util.stream.Stream;

public interface HasParts extends Document {
    String PROPERTY = "parts";
    
    default Stream<Part> getParts() {
        return children(PROPERTY, Part::new);
    }
}

In [6]:
/**
 * 定义小汽车类
 */

import java.util.Map;

public class Car extends AbstractDocument implements HasModel, HasPrice, HasParts {
    public Car(Map<String, Object> properties) {
        super(properties);
    }
}

In [7]:
/**
 * 一些单元测试
 */

import java.util.Arrays;
import java.util.HashMap;
import java.util.Map;
import java.util.stream.Stream;

public class App {
    static final PrintStream out = System.out;
    public static void main(String[] args) {
        Map<String, Object> carProps = new HashMap<>();
        carProps.put(HasModel.PROPERTY, "300SL");
        carProps.put(HasPrice.PROPERTY, 10000L);
        
        Map<String, Object> wheelProps = new HashMap<>();
        wheelProps.put(HasType.PROPERTY, "wheel");
        wheelProps.put(HasModel.PROPERTY, "15C");
        wheelProps.put(HasPrice.PROPERTY, 100L);
        
        Map<String, Object> doorProps = new HashMap<>();
        doorProps.put(HasType.PROPERTY, "door");
        doorProps.put(HasModel.PROPERTY, "Lambo");
        doorProps.put(HasPrice.PROPERTY, 300L);
        
        carProps.put(HasParts.PROPERTY, Arrays.asList(wheelProps, doorProps));
        
        Car car = new Car(carProps);
        out.println("Here is our car:");
        out.println("-> model: " + car.getModel().get());
        out.println("-> price: " + car.getPrice().get());
        out.println("-> parts: ");
        car.getParts().forEach(c -> out.println(String.format("\t%s/%s/%d", c.getType().get(),
            c.getModel().get(), c.getPrice().get())));
    }
}

In [8]:
App.main(null)

Here is our car:
-> model: 300SL
-> price: 10000
-> parts: 
	wheel/15C/100
	door/Lambo/300
